# Ejercicio: Calcular métricas con Scikit-Learn

## Contexto
El uso práctico de las métricas con Scikit-Learn permite validar modelos de forma profesional y precisa.

## Consigna
Utiliza Python y Scikit-Learn para calcular métricas de desempeño sobre un modelo ya entrenado (uno de regresión y uno de clasificación). Analiza los resultados y decide si el modelo es adecuado.

## Paso a paso

1. **Carga un dataset de ejemplo**:
   - Diabetes para regresión
   - Breast cancer para clasificación  
   (desde `sklearn.datasets`)

2. **Entrena un modelo básico**:
   - `LinearRegression` para regresión
   - `KNeighborsClassifier` para clasificación

3. **Calcula las siguientes métricas**:
   - **Regresión:** MAE, MSE, RMSE, R²
   - **Clasificación:** Precisión, Exactitud, Sensibilidad, Especificidad, AUC

4. **Muestra la matriz de confusión y la curva ROC** para clasificación.

5. **Interpreta** qué tan bien está funcionando cada modelo.


In [51]:
# Importar la librería de Colab para subir archivos
from google.colab import files

# Abrir un cuadro de selección para subir archivos
uploaded = files.upload()

# Mostrar los archivos subidos
for filename in uploaded.keys():
    print(f"Archivo subido: {filename}")


Saving clasificacion_dataset_fraude_tarjetas.csv to clasificacion_dataset_fraude_tarjetas (7).csv
Archivo subido: clasificacion_dataset_fraude_tarjetas (7).csv


In [52]:
import pandas as pd

# Leer CSV con codificación latin1 y separador ;
df = pd.read_csv('clasificacion_dataset_fraude_tarjetas.csv', encoding='latin1', sep=';')

# Mostrar las primeras 5 filas
df.head()


,Monto_transaccion,Hora,Pais,Comercio,Historial_compras,Fraude
0,4358.33,8,Brasil,Online,57,1
1,4867.58,0,USA,Supermercado,59,1
2,4844.54,4,Argentina,Supermercado,20,0
3,3749.51,19,Brasil,Restaurante,27,1
4,654.78,7,España,Supermercado,74,1


In [53]:
import pandas as pd

# Cargar CSV
df = pd.read_csv('clasificacion_dataset_fraude_tarjetas.csv', encoding='latin1', sep=';')

# 1️⃣ Revisar valores nulos
print("Valores nulos por columna:")
print(df.isnull().sum())

# 2️⃣ Revisar duplicados
print("\nNúmero de filas duplicadas:", df.duplicated().sum())

# 3️⃣ Opcional: eliminar duplicados si hay
df = df.drop_duplicates()
print("Filas después de eliminar duplicados:", df.shape[0])


Valores nulos por columna:
Monto_transaccion    0
Hora                 0
Pais                 0
Comercio             0
Historial_compras    0
Fraude               0
dtype: int64

Número de filas duplicadas: 0
Filas después de eliminar duplicados: 200


In [54]:
print(df.columns)


Index(['Monto_transaccion', 'Hora', 'Pais', 'Comercio', 'Historial_compras',
       'Fraude'],
      dtype='object')


In [55]:
df.head()


,Monto_transaccion,Hora,Pais,Comercio,Historial_compras,Fraude
0,4358.33,8,Brasil,Online,57,1
1,4867.58,0,USA,Supermercado,59,1
2,4844.54,4,Argentina,Supermercado,20,0
3,3749.51,19,Brasil,Restaurante,27,1
4,654.78,7,España,Supermercado,74,1


In [56]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt

# Convertir columnas categóricas a numéricas
df = pd.get_dummies(df, columns=['Pais', 'Comercio'])

# Separar características y target
X = df.drop('Fraude', axis=1)
y = df['Fraude']

# Entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Escalar variables numéricas
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Entrenar modelo KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Predecir
y_pred = knn.predict(X_test)
y_proba = knn.predict_proba(X_test)[:,1]

# Métricas
print("Exactitud:", accuracy_score(y_test, y_pred))
print("Precisión:", precision_score(y_test, y_pred))
print("Sensibilidad:", recall_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_proba))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))


Exactitud: 0.675
Precisión: 0.5
Sensibilidad: 0.46153846153846156
AUC: 0.7350427350427351
Matriz de confusión:
 [[21  6]
 [ 7  6]]


In [57]:
df.head()

,Monto_transaccion,Hora,Historial_compras,Fraude,Pais_Argentina,Pais_Brasil,Pais_Chile,Pais_España,Pais_USA,Comercio_Online,Comercio_Restaurante,Comercio_Retail,Comercio_Supermercado
0,4358.33,8,57,1,False,True,False,False,False,True,False,False,False
1,4867.58,0,59,1,False,False,False,False,True,False,False,False,True
2,4844.54,4,20,0,True,False,False,False,False,False,False,False,True
3,3749.51,19,27,1,False,True,False,False,False,False,True,False,False
4,654.78,7,74,1,False,False,False,True,False,False,False,False,True


##Conclusión:

El modelo KNN actual tiene exactitud moderada (67.5%), pero su sensibilidad es baja (46%), lo que significa que deja pasar casi la mitad de los fraudes reales. La precisión (50%) indica que muchas veces marca transacciones legítimas como fraude, y la AUC (0.735) muestra que el modelo distingue algo entre fraude y no fraude, pero no de manera óptima. La matriz de confusión confirma que hay falsos negativos y falsos positivos importantes, lo que limita la eficacia del modelo en detección de fraude.

##Sugerencia:

Para mejorar el modelo, se podría balancear las clases usando técnicas como SMOTE o ajustar el umbral de predicción, y probar modelos más robustos como Random Forest o XGBoost, que suelen manejar mejor datasets desbalanceados y aumentar la sensibilidad en la detección de fraude.

In [64]:
# Importar la librería de Colab para subir archivos
from google.colab import files

# Abrir un cuadro de selección para subir archivos
uploaded = files.upload()

# Mostrar los archivos subidos
for filename in uploaded.keys():
    print(f"Archivo subido: {filename}")


Saving regresion_dataset_consumo_energia.csv to regresion_dataset_consumo_energia (4).csv
Archivo subido: regresion_dataset_consumo_energia (4).csv


In [65]:
import pandas as pd

# Leer CSV con codificación latin1 y separador ;
df_2 = pd.read_csv('regresion_dataset_consumo_energia.csv', encoding='latin1', sep=';')

# Mostrar las primeras 5 filas
df_2.head()


,Temperatura,Hora_dia,Tipo_dia,Personas,Electrodomesticos,Consumo_kWh
0,22.5,1,Festivo,1,7,83.00
1,19.3,16,Laboral,4,7,87.32
2,23.2,7,Laboral,2,4,61.24
3,27.6,0,Festivo,2,3,60.33
4,18.8,8,Laboral,1,1,33.80


In [66]:
#Revisar valores nulos
print("Valores nulos por columna:")
print(df_2.isnull().sum())

#Revisar duplicados
print("\nNúmero de filas duplicadas:", df_2.duplicated().sum())

#Eliminar duplicados si hay
df_2 = df_2.drop_duplicates()
print("Filas después de eliminar duplicados:", df_2.shape[0])


Valores nulos por columna:
Temperatura          0
Hora_dia             0
Tipo_dia             0
Personas             0
Electrodomesticos    0
Consumo_kWh          0
dtype: int64

Número de filas duplicadas: 0
Filas después de eliminar duplicados: 200


In [67]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Definir X e y para df_2
X = df_2.drop("Consumo_kWh", axis=1)
y = df_2["Consumo_kWh"]

# Codificar variable categórica
X = pd.get_dummies(X, drop_first=True)

# División en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Predicciones
y_pred = model.predict(X_test)

# Evaluación
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R²:", r2)


MAE: 4.460495889378419
MSE: 32.26168734703726
RMSE: 5.67993726611811
R²: 0.8822208185759872


##Conclusión:
El modelo de regresión lineal predice el consumo eléctrico bastante bien, con un R² de 0.88, lo que significa que explica el 88% de la variabilidad en los datos. Los errores promedio (MAE ≈ 4.46 kWh, RMSE ≈ 5.68 kWh) son relativamente bajos considerando el rango de consumo del dataset. Esto indica que el modelo captura la mayor parte de la tendencia, aunque hay algunos desvíos puntuales.

##Sugerencias de mejora:

Explorar relaciones no lineales entre variables, por ejemplo mediante Random Forest o Gradient Boosting.

Realizar feature engineering, como crear categorías de hora del día o rangos de temperatura.

Escalar variables numéricas para mejorar la estabilidad del modelo.

Evaluar posibles interacciones entre variables (por ejemplo, número de personas × electrodomésticos).